# Load Fused feature

In [ ]:
def load_feature(feat_name):
  import numpy as np
  from sklearn.model_selection import train_test_split

  feature = np.load('/fusedfeat/'+feat_name+'.npy')
  labelid = np.load('/dataset/'+feat_name+'/label_75.npy')

In [ ]:
  X_train, X_test, y_train, y_test = train_test_split(feature, labelid, test_size=0.25, random_state=42)
  return X_train, X_test, y_train, y_test

# Create Model

In [6]:
def create_model():
    from tensorflow.keras.models import Model, Sequential
    from tensorflow.keras.layers import Dense, Input, Concatenate,Conv3D,ZeroPadding3D,TimeDistributed
    from tensorflow.keras.regularizers import l2
    from tensorflow.python.keras.layers.pooling import GlobalAveragePooling3D,MaxPooling3D,GlobalMaxPooling3D
    from tensorflow.python.keras.layers.recurrent import LSTM
    #from tensorflow.python.keras.layers import TimeDistributed
    from tensorflow.keras.layers import GlobalAveragePooling2D,BatchNormalization,Dropout


    input_layer = Input(shape=(16,7,7,2048))
    encoded_sequence = BatchNormalization()(input_layer)
    encoded_sequence = Conv3D(1024, kernel_size=(1, 2, 2),strides=1, activation='relu')(encoded_sequence)
    encoded_sequence = BatchNormalization()(encoded_sequence)
    encoded_sequence = Dropout(0.2)(encoded_sequence)
    encoded_sequence = GlobalAveragePooling3D()(encoded_sequence)
    hidden_layer = Dense(2048, activation="relu")(encoded_sequence)
    outputs = Dense(2, activation='softmax')(hidden_layer)
    model = Model(input_layer, outputs)

    return model

# Train Model

In [ ]:
def train_model(feat_name,learningrate,batchsize):
  from tensorflow.keras.optimizers import SGD
  from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
  import numpy as np
  X_train, X_test, y_train, y_test = load_feature(feat_name)
  model = create_model()
  optimizer_new=SGD(lr=lr, momentum=0.9, nesterov=True)
  model.compile(optimizer=optimizer_new,loss='binary_crossentropy',metrics=['acc'])
  checkpointer = ModelCheckpoint(filepath='/3d_model/model'+feat_name+'.hdf5', verbose=1, save_best_only=True)
  history_new_cnlst=model.fit(X_train,y_train,batch_size=bz,epochs=500,validation_data=(X_test,y_test),callbacks=[checkpointer])
  _, test_acc_cv = model.evaluate(X_test, y_test, verbose=0)
  return test_acc_cv

In [ ]:
import numpy as np
feat_name = ['hockey_features']
learningrate = [0.01,0.001,0.0001]
batchsize = [4,8]
for feat in feat_name:
  for bz in batchsize:
    for lr in learningrate:
      acc,t = train_model(feat,lr,bz)

feat_name = ['movie_features']
learningrate = [0.01,0.001,0.0001]
batchsize = [4,8]
for feat in feat_name:
  for bz in batchsize:
    for lr in learningrate:
      acc,t = train_model(feat,lr,bz)

feat_name = ['violentflow_features']
learningrate = [0.01,0.001,0.0001]
batchsize = [4,8]
for feat in feat_name:
  for bz in batchsize:
    for lr in learningrate:
      acc,t = train_model(feat,lr,bz)

feat_name = ['all_features']
learningrate = [0.01,0.001,0.0001]
batchsize = [4,8]
for feat in feat_name:
  for bz in batchsize:
    for lr in learningrate:
      acc,t = train_model(feat,lr,bz)